In [1]:
import foursquare as fs
import pandas as pd
import time

In [2]:
client_id = 'your_client_id'
client_secret = 'your_client_secret'
redirect_id = 'site_of_your_app'

client = fs.Foursquare(client_id=client_id, client_secret=client_secret, 
                       redirect_uri=redirect_id)

In [3]:
# returns a list of venues in a given area
def venues_list(client, ne, sw):
    
    # see also https://developer.foursquare.com/categorytree
    categories = '4d4b7105d754a06374d81259,4d4b7105d754a06376d81259'
    
    vens = client.venues.search({'intent' : 'browse', 'limit' : 1000,
                                 'ne' : ne, 'sw' : sw,
                                 'categoryId' : categories})
    
    return [venue['id'] for venue in vens['venues']]

In [4]:
# returns a pandas DF that contains information about
# Discrict (that you choose), BarId, UserId, Review text,
# whether a visitor made a photo, and visitor's reaction
def reviews(client, VENUE_ID_LIST, district_name):
    
    df = pd.DataFrame(columns=['District', 'BarId', 'UserId', 'Review', 
                               'MadePhoto', 'Reaction'])
    
    for VENUE_ID in VENUE_ID_LIST:
        reviews = client.venues.tips(VENUE_ID, params={'limit':1000, 
                                                       'sort':'popular'})
        
        for review in reviews['tips']['items']:
            UserId = review['user']['id']
            MadePhoto = False if review.get('photo', 0) == 0 else True
            Review = review['text']
            Reaction = review.get('authorInteractionType', None)
            
            df = df.append([{'District' : district_name, 
                             'BarId' : VENUE_ID, 'UserId' : UserId, 
                             'Review' : Review, 'MadePhoto' : MadePhoto,
                             'Reaction' : Reaction}])
        
    return df

In [5]:
# specifying coordinates of districts
# Note: only 50 venues will be reported within chosen area 
# in functions I choose some atbitrary large constant --- 'limit' : 1000
coords = {'sw' : '59.929506,30.320345', 'ne' : '59.935408,30.334722'}
venues = venues_list(client, ne=coords['ne'], sw=coords['sw'])

In [6]:
# downloading reviews for districts
data = reviews(client, VENUE_ID_LIST=venues, district_name='dist1')

In [7]:
# example of received VenueIDs
venues[:5]

['52b89801498eb5da45f72173',
 '4c62e1907c9def3bb65ad71c',
 '4fdc23abe4b019d606ac005d',
 '531092cd11d271858db84e20',
 '4d88a71b4757721ed4a09169']

In [8]:
# example of 
data.head()

,District,BarId,UserId,Review,MadePhoto,Reaction
0,dist1,52b89801498eb5da45f72173,142661708,"Как в городе с хорошими локальными, интересным...",False,disliked
0,dist1,52b89801498eb5da45f72173,78572773,Сегодня помогли выбрать термос-кружку! Очень в...,True,liked
0,dist1,52b89801498eb5da45f72173,27549866,"Уютно, красивый вид на Казанскую площадь и баш...",True,None
0,dist1,52b89801498eb5da45f72173,14572902,"Первый в городе старбакс, действительно похожи...",True,None
0,dist1,52b89801498eb5da45f72173,51792004,"Самый лучший Starbucks, на мой взгляд. Интерес...",True,liked
